# AI-FW2 効果測定1 問題2

google colaboratoryに"kouka_1_2_data.zip"をアップロードし、下記のセルを実行してデータを解凍してください

In [ ]:
!unzip "kouka_1_2_data.zip"

In [2]:
# ライブラリのインポート 
import cv2
import matplotlib.pyplot as plt
import pickle

四角枠の領域を抽出する

In [3]:
# 画像から四角の領域を抽出する関数を作成する
# * 引数 fname ... 読み込む画像のファイル名
# * 戻り値1 ... 画像の中にある矩形の座標と大きさ(x座標, y座標, 幅, 高さ)のリスト
# * 戻り値2 ... 画像イメージ
def detect_rectangle(fname):
    # fnameの画像を読み込む
    img = cv2.imread(fname)

    # 画像を二値化
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0) 
    im2 = cv2.threshold(gray, 140, 255, cv2.THRESH_BINARY_INV)[1]
    
    # 輪郭を抽出
    cnts = cv2.findContours(im2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]


    # 抽出した輪郭を単純なリストに変換
    result = []
    for pt in cnts:
        x, y, w, h = cv2.boundingRect(pt)

        # 大きすぎたり小さすぎたりする領域を除去
        # 今回は幅が50以下、70以上のものは格納しない
        if not(50 < w < 70): continue
        result.append([x, y, w, h])

    # 抽出した輪郭のソートや、近すぎる領域の除去は行わず、結果を返す
    return result, img

四角枠の領域の数値を判別する

In [4]:
# 学習済みの手書き数字認識モデル「digits.pkl」を読み込む
with open('digits.pkl', 'rb') as fp:
    clf = pickle.load(fp)

In [5]:
# 四角枠内の数値を識別し、リストを返す関数を作成する
# * 引数 fname ... 読み込む画像のファイル名
# * 戻り値 ... 認識した数値のリスト
def predict_numbers(fname):
    # 返却用リストの初期化
    result = []

    # 画像から四角枠の領域を読み込む
    cnts, img = detect_rectangle(fname)

    # 読み込んだデータに含まれている数値を検出し、リストに追加する
    for i, pt in enumerate(cnts):
        x, y, w, h = pt

        # 枠線の輪郭分だけ小さくする
        x += 8
        y += 8
        w -= 16
        h -= 16

        # 画像データを取り出す
        im2 = img[y:y+h, x:x+w]

        # 画像データを学習済みデータに合わせる
        # グレイスケールに変換
        im2gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
        # リサイズ
        im2gray = cv2.resize(im2gray, (8, 8))
        # 白黒反転 
        im2gray = 15 - im2gray // 16 
        # 一次元に変換
        im2gray = im2gray.reshape((-1, 64)) 

        #データを予測する
        res = clf.predict(im2gray)

        # 予測結果をresultに追加する
        result.append(res[0])

    # 識別結果の返却
    return result

ターゲット数値を取得する

In [6]:
# 以下コメント以外は空欄にする

# 作成したpredict_numbers関数を利用し、調査ターゲットの画像の判別結果を取得する
res = predict_numbers('target.png')

# 調査ターゲットの数値を集合に代入する
target_nums = set(res)

# 調査ターゲットの数値を表示する
print(target_nums)

{1, 2, 3}



チェック対象の画像にターゲット数値が含まれるかチェックする

In [7]:
filenames = ['check1.png','check2.png','check3.png','check4.png','check5.png']

for filename in filenames:

  # 画像を読み込み、認識結果を取得する
  result_set = set(predict_numbers(filename))

  # ターゲット数値が含まれていれば、どのターゲット数値が含まれていたかを表示する
  # ターゲット数値が含まれていなければ、「含まれない」と表示する
  check_result = target_nums & result_set
  if len(check_result) > 0:
      print('ファイル名：{} 含まれたターゲット数値：{}'.format(filename,check_result))
  else:
      print('ファイル名：{} 含まれない'.format(filename))

ファイル名：check1.png 含まれたターゲット数値：{3}
ファイル名：check2.png 含まれない
ファイル名：check3.png 含まれたターゲット数値：{1}
ファイル名：check4.png 含まれたターゲット数値：{2}
ファイル名：check5.png 含まれたターゲット数値：{1, 2}


▶実行結果  
ファイル名：check1.png 含まれたターゲット数値：{3}  
ファイル名：check2.png 含まれない  
ファイル名：check3.png 含まれたターゲット数値：{1}  
ファイル名：check4.png 含まれたターゲット数値：{2}  
ファイル名：check5.png 含まれたターゲット数値：{1, 2}